In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
sys.path.append('../../src')

from common import common, transform, json
from common.json import IA_FORMAT
from individual_activity import main as ia
from individual_activity.individual_activity import IndividualActivity

In [2]:
room_num = '09'
date = '20210304'
name = 'pass1'

In [3]:
individual_activity_json = os.path.join(
    common.data_dir, '{0}/{1}/{2}/json/individual_activity.json'.format(room_num, date, name))
field_path = os.path.join(
    common.data_dir, '{}/field0304.png'.format(room_num))

In [4]:
# homography
field_raw = cv2.imread(field_path)
p_video = transform.homo[room_num + '_0304'][0]
p_field = transform.homo[room_num + '_0304'][1]
homo = transform.Homography(p_video, p_field, field_raw.shape)

In [5]:
individual_activity_data = json.load(individual_activity_json)

In [6]:
def make_individuals(datas, homo):
    individuals = {}
    for data in datas:
        label = data[IA_FORMAT[0]]
        frame_num = data[IA_FORMAT[1]]
        tracking_point = data[IA_FORMAT[2]]

        if label not in individuals:
            ia = IndividualActivity(label, homo)
            individuals[label] = ia
        else:
            ia = individuals[label]

        ia.tracking_points[frame_num] = tracking_point
        for f in IA_FORMAT[3:]:
            ia.indicator_dict[f][frame_num] = data[f]
            
    return individuals

In [7]:
individuals = make_individuals(individual_activity_data, homo)

In [ ]:
for idv in individuals:
    for frame_num, pt in idv.tracking_points.items():